###### 2020-11-20 금요일

# 01_Bidirect LSTM

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import pandas as pd
import numpy as np
import re

from tqdm import tqdm_notebook

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import accuracy_score, log_loss, confusion_matrix
from sklearn.model_selection import StratifiedKFold, train_test_split

## 1. 데이터불러오기 & 합치기
 - 데이터의 특징은 다음과 같다
    - 이모티콘, 특수문자는 제거
    - 0 : 악플이 아닌경우
    - 1 : 특정인을 비방하지는 않지만, 일부 소수에게 불쾌감을 주거나 악플여부를 매기기 애매한 경우
    - 2 : 대부분의 사람이 불편함을 느끼는 댓글
    

In [ ]:
d1 = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/data/division_data1(2020-11-17).csv')
d2 = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/data/division_data2(2020-11-17).csv')
# division_data3 경우는 인코딩안해주면 에러뜸 ㅠ
d3 = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/data/division_data3(2020-11-17).csv', encoding='cp949')
# division_data4 경우는 read_excel로 불러와야 에러안뜸 ㅠ
d5 = pd.read_excel('/content/drive/MyDrive/[final_project]_악플원정대/data/division_data5(2020-11-17).csv')
d6 = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/data/division_data6(재원).csv')

In [ ]:
df_list = [d1, d2, d3, d5, d6]

all_df = d1
for df in df_list[1:]:
    all_df = pd.concat([all_df, df])

## 2. 데이터전처리

In [ ]:
all_df.drop(['Unnamed: 0', '출처'], axis=1, inplace=True)

In [ ]:
index = all_df['악플여부'].isna()
raw_df = all_df[~index]

In [ ]:
raw_df = raw_df.reset_index()
raw_df.drop(['index'], inplace=True, axis=1)

In [ ]:
copy_df = raw_df.copy()

In [ ]:
# 고정 시드값 지정
seed = 123

# 댓글 길이 지정
comment_len = 400

In [ ]:
copy_df['악플여부'].value_counts()

In [ ]:
copy_df['댓글'] = copy_df['댓글'].apply(str)

## 3. Train/Test나누기 & 텍스트 음절단위 토큰화 & 정수인덱싱 & 패딩

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split


### Train/Test 나누기

In [ ]:
feature = copy_df['댓글']
label = copy_df['악플여부']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feature, label, 
                                                    test_size=0.2,
                                                    random_state=seed)

### 텍스트 음절단위 토큰화
 - 조금 오래... 걸립니다..ㅎ

In [ ]:
X_train_split = X_train.apply(list).tolist()
X_test_split = X_test.apply(list).tolist()

X_train_token_list =  sum(X_train_split, [])

### 정수인덱싱

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_token_list)

word_index_vocab = tokenizer.word_index
word_count_vocab = tokenizer.word_counts

X_train_sequences = tokenizer.texts_to_sequences(X_train_split)
X_test_sequences = tokenizer.texts_to_sequences(X_test_split)



### 패딩

In [ ]:
train = pad_sequences(X_train_sequences, padding='post', maxlen=400)
test = pad_sequences(X_test_sequences, padding='post', maxlen=400)

## 4. SMOTE를 이용한 라벨 불균형 해소

In [ ]:
y_train.value_counts()

 - 라벨 0은 446162, 1은 2897, 2는 4993으로 균형이 잡혀있지않아 recall(재현율)이 떨어질 가능성이 높다
 - 그래서 SMOTE를 이용하여 라벨이 1과 2인 경우를 복제하여 라벨학습의 균형을 맞추어 주는 것이다
 - 이것을 `오버샘플링`이라 한다.

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote = SMOTE(random_state=seed)
train_over, y_train_over = smote.fit_sample(train, y_train)

In [ ]:
pd.Series(y_train_over).value_counts()

## 5. Bidirect LSTM 구현

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import f1_score

In [ ]:
# 음절 단어사전의 크기
vocab_size = len(word_index_vocab)
embedding_dim = 64
comment_len = 400

In [ ]:
model = Sequential()
model.add(Embedding((vocab_size)+1, embedding_dim, input_length=comment_len))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(32, return_sequences=False)))
model.add(Dense(3, activation='softmax'))


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
# 성능의 변화가 없을때 멈추는 기능
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)

# 지금까지 가장 좋은 성능이 나왔을때, 노드의 가중치를 저장하는 함수
mc = ModelCheckpoint('best_model.h5', monitor= 'val_acc', mode='max', save_best_only=True)

In [ ]:
history = model.fit(train_over, y_train_over, 
                    callbacks        = [es, mc],
                    epochs           = 5, 
                    batch_size       = 128, 
                    validation_split = 0.2)

## 6. Test Set 예측
 - confusion matrix의 대각선 원소의 수가 많아지도록 모델을 짜주십쇼

In [ ]:
loaded_model = load_model('best_model.h5')
y_pred = loaded_model.predict(test)

y_pred_class = np.argmax(y_pred)
confusion_matrix(y_pred_class, y_test)